Compare times between the test cases run using Python/C code (int2e-speed.ipynb) and Julia code. Here's the Julia code from pyquante.jl:

In [19]:
function gaussian_product_center(aexpn::Float64,ax::Float64,ay::Float64,az::Float64,
                                    bexpn::Float64,bx::Float64,by::Float64,bz::Float64)
    return (aexpn*[ax,ay,az]+bexpn*[bx,by,bz])/(aexpn+bexpn)    
end
dist2(dx,dy,dz) = dx*dx+dy*dy+dz*dz # Is there something in the standard library that does this?
function Fgamma(m::Int64,x::Float64,SMALL::Float64=1e-12)
    #println("Fgamma($m,$x)")
    x = max(x,SMALL) # Evidently needs underflow protection
    return 0.5*x^(-m-0.5)*gammainc(m+0.5,x)
end

function gammainc(a::Float64,x::Float64)
    # This is the series version of gamma from pyquante. For reasons I don't get, it 
    # doesn't work around a=1. This works alright, but is only a stopgap solution
    # until Julia gets an incomplete gamma function programmed
    if abs(a-1) < 1e-3
        println("Warning: gammainc_series is known to have problems for a ~ 1")
    end
    if x < (a+1.0)
        #Use the series representation
        gam,gln = gser(a,x)
    else 
        #Use continued fractions
        gamc,gln = gcf(a,x)
        gam = 1-gamc
    end
    return exp(gln)*gam
end

function gser(a::Float64,x::Float64,ITMAX::Int64=100,EPS::Float64=3e-9)
    # Series representation of Gamma. NumRec sect 6.1.
    gln=lgamma(a)
    if x == 0
        return 0,gln
    end
    ap = a
    delt = s = 1/a
    for i in 1:ITMAX
        ap += 1
        delt *= (x/ap)
        s += delt
        if abs(delt) < abs(s)*EPS
            break
        end
    end
    return s*exp(-x+a*log(x)-gln),gln
end

function gcf(a::Float64,x::Float64,ITMAX::Int64=200,EPS::Float64=3e-9,FPMIN::Float64=1e-30)
    #Continued fraction representation of Gamma. NumRec sect 6.1"
    gln=lgamma(a)
    b=x+1.-a
    c=1./FPMIN
    d=1./b
    h=d
    for i in 1:ITMAX
        an=-i*(i-a)
        b=b+2.
        d=an*d+b
        if abs(d) < FPMIN
            d=FPMIN
        end
        c=b+an/c
        if abs(c) < FPMIN
            c=FPMIN
        end
        d=1./d
        delt=d*c
        h=h*delt
        if abs(delt-1.) < EPS
            break
        end
    end
    gammcf = exp(-x+a*log(x)-gln)*h
    return gammcf,gln
end

gcf (generic function with 4 methods)

In [20]:
function vrr(aexpn::Float64,ax::Float64,ay::Float64,az::Float64,aI::Int64,aJ::Int64,aK::Int64,
        bexpn::Float64,bx::Float64,by::Float64,bz::Float64,
        cexpn::Float64,cx::Float64,cy::Float64,cz::Float64,cI::Int64,cJ::Int64,cK::Int64,
        dexpn::Float64,dx::Float64,dy::Float64,dz::Float64,m::Int64)
    px,py,pz = gaussian_product_center(aexpn,ax,ay,az,bexpn,bx,by,bz)
    qx,qy,qz = gaussian_product_center(cexpn,cx,cy,cz,dexpn,dx,dy,dz)
    zeta,eta = aexpn+bexpn,cexpn+dexpn
    wx,wy,wz = gaussian_product_center(zeta,px,py,pz,eta,qx,qy,qz)
    #println("P: $px,$py,$pz, Q: $qx,$qy,$qz, W: $wx,$wy,$wz, $zeta,$eta")
    
    val = 0
    if cK>0
        val = (qz-cz)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
            cexpn,cx,cy,cz,cI,cJ,cK-1,dexpn,dx,dy,dz,m) +
            (wz-qz)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK-1,dexpn,dx,dy,dz,m+1)
        #println("val1=$val")
        if cK>1
            val += 0.5*(cK-1)/eta*(
                vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                    cexpn,cx,cy,cz,cI,cJ,cK-2,dexpn,dx,dy,dz,m) -
            zeta/(zeta+eta)*
                vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                    cexpn,cx,cy,cz,cI,cJ,cK-2,dexpn,dx,dy,dz,m+1) )
        #println("val2=$val")
        end
        if aK>0
            val += 0.5*aK/(zeta+eta)*
                vrr(aexpn,ax,ay,az,aI,aJ,aK-1,bexpn,bx,by,bz,
                    cexpn,cx,cy,cz,cI,cJ,cK-1,dexpn,dx,dy,dz,m+1)
        end
        #println("val3=$val")
        return val
    elseif cJ>0
        val = (qy-cy)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
            cexpn,cx,cy,cz,cI,cJ-1,cK,dexpn,dx,dy,dz,m) +
        (wy-qy)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ-1,cK-1,dexpn,dx,dy,dz,m+1)
        #println("val4=$val")
        if cJ>1
            val += 0.5*(cJ-1)/eta*(
            vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ-2,cK,dexpn,dx,dy,dz,m) -
            zeta/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ-2,cK,dexpn,dx,dy,dz,m+1)
            )
        #println("val5=$val")
        end
        if aJ>0
            val += 0.5*aJ/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI,aJ-1,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ-1,cK,dexpn,dx,dy,dz,m+1)
        end
        #println("val6=$val")
        return val
    elseif cI>0
        val = (qx-cx)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
            cexpn,cx,cy,cz,cI-1,cJ,cK,dexpn,dx,dy,dz,m) +
        (wx-qx)*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-1,cJ,cK-1,dexpn,dx,dy,dz,m+1)
        #println("val7=$val")
        if cI>1
            val += 0.5*(cI-1)/eta*(
            vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-2,cJ,cK,dexpn,dx,dy,dz,m) -
            zeta/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-2,cJ,cK,dexpn,dx,dy,dz,m+1)
            )
        end
        #println("val8=$val")
        if aI>0
            val += 0.5*aI/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI-1,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-1,cJ,cK,dexpn,dx,dy,dz,m+1)
        end
        #println("val9=$val")
        return val
    elseif aK>0
        val = (pz-az)*vrr(aexpn,ax,ay,az,aI,aJ,aK-1,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m) +
        (wz-pz)*vrr(aexpn,ax,ay,az,aI,aJ,aK-1,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m+1)
        #println("val10=$val")
        if aK>1
            val += 0.5*(aK-1)/zeta*(
            vrr(aexpn,ax,ay,az,aI,aJ,aK-2,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-1,cJ,cK,dexpn,dx,dy,dz,m) -
            eta/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI,aJ,aK-2,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI-1,cJ,cK,dexpn,dx,dy,dz,m+1)
            )
        end
        #println("val11=$val")
        return val
    elseif aJ>0
        val = (py-ay)*vrr(aexpn,ax,ay,az,aI,aJ-1,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m)+
        (wy-py)*vrr(aexpn,ax,ay,az,aI,aJ-1,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m+1)
        #println("val12=$val")
        if aJ>1
            val += 0.5*(aJ-1)/zeta*(
            vrr(aexpn,ax,ay,az,aI,aJ-2,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m) -
            eta/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI,aJ-2,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m+1)
            )
        end
        #println("val13=$val")
        return val
    elseif aI>0
        val = (px-ax)*vrr(aexpn,ax,ay,az,aI-1,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m) +
        (wx-px)*vrr(aexpn,ax,ay,az,aI-1,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m+1)
        #println("val14=$val")
        if aI>1
            val += 0.5*(aI-1)/zeta*(
            vrr(aexpn,ax,ay,az,aI-2,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m) -
            eta/(zeta+eta)*
            vrr(aexpn,ax,ay,az,aI-2,aJ,aK,bexpn,bx,by,bz,
                cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,m+1)
            )
        end
        #println("val15=$val")
        return val
    end

    rab2 = dist2(ax-bx,ay-by,az-bz)
    rcd2 = dist2(cx-dx,cy-dy,cz-dz)
    rpq2 = dist2(px-qx,py-qy,pz-qz)
    T = zeta*eta/(zeta+eta)*rpq2
    Kab = sqrt(2)pi^1.25/zeta*exp(-aexpn*bexpn*rab2/zeta)
    Kcd = sqrt(2)pi^1.25/eta*exp(-cexpn*dexpn*rcd2/eta)
    #println("rab2=$rab2,rcd2=$rcd2,rpq2=$rpq2,T=$T,Kab=$Kab,Kcd=$Kcd")
    return Kab*Kcd/sqrt(zeta+eta)*Fgamma(m,T)
end

vrr (generic function with 1 method)

In [21]:
function vrr_iter(aexpn::Float64,ax::Float64,ay::Float64,az::Float64,aI::Int64,aJ::Int64,aK::Int64,
        bexpn::Float64,bx::Float64,by::Float64,bz::Float64,
        cexpn::Float64,cx::Float64,cy::Float64,cz::Float64,cI::Int64,cJ::Int64,cK::Int64,
        dexpn::Float64,dx::Float64,dy::Float64,dz::Float64,M::Int64)
    px,py,pz = gaussian_product_center(aexpn,ax,ay,az,bexpn,bx,by,bz)
    qx,qy,qz = gaussian_product_center(cexpn,cx,cy,cz,dexpn,dx,dy,dz)
    zeta,eta = aexpn+bexpn,cexpn+dexpn
    wx,wy,wz = gaussian_product_center(zeta,px,py,pz,eta,qx,qy,qz)
    rab2 = dist2(ax-bx,ay-by,az-bz)
    rcd2 = dist2(cx-dx,cy-dy,cz-dz)
    rpq2 = dist2(px-qx,py-qy,pz-qz)
    T = zeta*eta/(zeta+eta)*rpq2
    Kab = sqrt(2)pi^1.25/zeta*exp(-aexpn*bexpn*rab2/zeta)
    Kcd = sqrt(2)pi^1.25/eta*exp(-cexpn*dexpn*rcd2/eta)
    mtot = aI+aJ+aK+cI+cJ+cK+M

    vrr_terms = zeros(Float64,(aI+1,aJ+1,aK+1,cI+1,cJ+1,cK+1,mtot+1))
    
    for m in 0:mtot
        vrr_terms[1,1,1, 1,1,1, m+1] = Fgamma(m,T)*Kab*Kcd/sqrt(zeta+eta)
    end
    
    for i in 0:(aI-1)
        for m in 0:(mtot-i-1)
            vrr_terms[i+2,1,1, 1,1,1, m+1] = (
                 (px-ax)*vrr_terms[i+1,1,1, 1,1,1, m+1] + 
                 (wx-px)*vrr_terms[i+1,1,1, 1,1,1, m+2])

            if i>0
                vrr_terms[i+2,1,1, 1,1,1, m+1] += i/2/zeta*(
                    vrr_terms[i,1,1, 1,1,1, m+1] -
                    eta/(zeta+eta)*vrr_terms[i,1,1, 1,1,1, m+2])
            end
        end
    end
#=
    for j in 0:(aJ-1)
        for i in 0:aI
            for m in 0:(mtot-i-j-1)
                println(("b",i,j,m))
                vrr_terms[i+1,j+2,1, 1,1,1, m+1] = (
                (py-ay)*vrr_terms[i+1,j+1,1, 1,1,1, m+1] +
                (wy-py)*vrr_terms[i+1,j+1,1, 1,1,1, m+2])
                if j>0
                    vrr_terms[i+1,j+2,1, 1,1,1, m+1] += j/2/zeta*(
                        vrr_terms[i+1,j,1, 1,1,1, m+1] -
                        eta/(zeta+eta)*vrr_terms[i+1,j,1, 1,1,1, m+2])
                end
            end
        end
    end

    for k in 0:(aK-1)
        for j in 0:aJ
            for i in 0:aI
                for m in 0:(mtot-i-j-k-1)
                    println(("c",i,j,k,m))
                    vrr_terms[i+1,j+1,k+2, 1,1,1, m+1] = (
                    (pz-az)*vrr_terms[i+1,j+1,k+1, 1,1,1, m+1] +
                    (wz-pz)*vrr_terms[i+1,j+1,k+1, 1,1,1, m+2])
                    if k>0
                        vrr_terms[i+1,j+1,k+2, 1,1,1, m+1] += k/2/zeta*(
                        vrr_terms[i+1,j+1,k, 1,1,1, m+1] -
                        eta/(zeta+eta)*vrr_terms[i+1,j+1,k, 1,1,1, m+2])
                    end
                end
            end
        end
    end

    for q in 0:(cI-1)
        for k in 0:aK
            for j in 0:aJ
                for i in 0:aI
                    for m in 0:(mtot-i-j-k-q-1)
                        println(("d",i,j,k,q,m))
                        vrr_terms[i+1,j+1,k+1, q+2,1,1, m+1] = (
                        (qx-cx)*vrr_terms[i+1,j+1,k+1, q+1,1,1, m+1] +
                        (wx-qx)*vrr_terms[i+1,j+1,k+1, q+1,1,1, m+2])
                        if q>0
                            vrr_terms[i+1,j+1,k+1, q+2,1,1, m+1] += q/2/eta*(
                            vrr_terms[i+1,j+1,k+1, q,1,1, m+1] -
                            eta/(zeta+eta)*vrr_terms[i+1,j+1,k+1, q,1,1, m+2])
                        end
                        if i>0
                            vrr_terms[i+1,j+1,k+1, q+2,1,1, m+1] += (
                            i/2/(zeta+eta)*vrr_terms[i,j+1,j+1, q+1,1,1, m+2])
                        end
                    end
                end
            end
        end
    end

    for r in 0:(cJ-1)
        for q in 0:cI
            for k in 0:aK
                for j in 0:aJ
                    for i in 0:aI
                        for m in 0:(mtot-i-j-k-q-r-1)
                            println(("e",i,j,k,q,r,m))
                            vrr_terms[i+1,j+1,k+1, q+1,r+2,1, m+1] = (
                            (qy-cy)*vrr_terms[i+1,j+1,k+1, q+1,r+1,1, m+1] +
                            (wy-qy)*vrr_terms[i+1,j+1,k+1, q+1,r+1,1, m+2])
                            if r>0
                                vrr_terms[i+1,j+1,k+1, q+1,r+2,1, m+1] += r/2/eta*(
                                vrr_terms[i+1,j+1,k+1, q+1,r+1,1, m+1] -
                                zeta/(zeta+eta)*vrr_terms[i+1,j+1,k+1, q+1,r,1, m+2])
                            end
                            if j>0
                                vrr_terms[i+1,j+1,k+1, q+1,r+2,1, m+1] += (
                                j/2/(zeta+eta)*vrr_terms[i+1,j,k+1, q+1,r+1,1, m+2])
                            end
                        end
                    end
                end
            end
        end
    end

    for s in 0:(cK-1)
        for r in 0:cJ
            for q in 0:cI
                for k in 0:aK
                    for j in 0:aJ
                        for i in 0:aI
                            for m in 0:(mtot-i-j-k-q-r-s-1)
                                println(("f",i,j,k,q,r,s,m))
                                vrr_terms[i+1,j+1,k+1, q+1,r+1,s+2, m+1] = (
                                (qz-cz)*vrr_terms[i+1,j+1,k+1, q+1,r+1,s+1, m+1]+
                                (wz-qz)*vrr_terms[i+1,j+1,k+1, q+1,r+1,s+1, m+2])
                                if s>0
                                    vrr_terms[i+1,j+1,k+1, q+1,r+1,s+2,m+1] += s/2/eta*(
                                    vrr_terms[i+1,j+1,k+1,q+1,r+1,s, m+1] -
                                    zeta/(zeta+eta)*vrr_terms[i+1,j+1,k+1,q+1,r+1,s,m+2])
                                end
                                if k>0
                                    vrr_terms[i+1,j+1,k+1, q+1,r+1,s+2,m+1] += (
                                    k/2/(zeta+eta)*vrr_terms[i+1,j+1,k, q+1,r+1,s+1,m+2])
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    println("before return")
    =#
    vrr_terms[aI+1,aJ+1,aK+1,cI+1,cJ+1,cK+1,M+1]
end

vrr_iter (generic function with 1 method)

In [24]:
function test_vrr()
    ax=ay=az=bx=by=bz=cx=cy=cz=dx=dy=dz=0.0
    aexpn=bexpn=cexpn=dexpn=1.0
    aI=aJ=aK=0
    cI=cJ=cK=0
    M=0

    for (ax,ay,az, aI,aJ,aK, cI,cJ,cK, result) in [
            (0.,0.,0., 0,0,0, 0,0,0, 4.37335456733),
            (0.,0.,0., 1,0,0, 1,0,0, 0.182223107579),
            (0.,0.,0., 0,1,0, 0,1,0, 0.182223107579),
            (0.,0.,0., 0,0,1, 0,0,1, 0.182223107579),

            (0.,0.,0., 2,0,0, 2,0,0,  0.223223306785),
            (0.,0.,0., 0,2,0, 0,2,0,  0.223223306785),
            (0.,0.,0., 0,0,2, 0,0,2,  0.223223306785),

            (1.,2.,3., 1,0,0, 1,0,0, -5.63387712455e-06),
            (1.,2.,3., 0,1,0, 0,1,0, -0.000116463120359),
            (1.,2.,3., 0,0,1, 0,0,1, -0.000301178525749),

            (1.,2.,3., 2,0,0, 2,0,0, 0.00022503308545040895),
            (1.,2.,3., 0,2,0, 0,2,0, 0.0006102470883881907),
            (1.,2.,3., 0,0,2, 0,0,2, 0.0013427831014563411),

            (0.,0.,0., 1,1,0, 1,1,0, 0.0136667330685),
            (0.,0.,0., 0,1,1, 0,1,1, 0.0136667330685),
            (0.,0.,0., 1,0,1, 1,0,1, 0.0136667330685),

            (3.,2.,1., 1,1,0, 1,1,0, 5.976771621486971e-5),
            (3.,2.,1., 0,1,1, 0,1,1, 1.5742904443905067e-6),
            (3.,2.,1., 1,0,1, 1,0,1, 4.00292848649699e-6)
        ]

        val1 = vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
            cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,M)
        val2 = vrr(cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,
            aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,M)
        @assert isapprox(val1,val2)
        @assert isapprox(val1,result)
        val3 = vrr_iter(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
            cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,M)
        val4 = vrr_iter(cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,
            aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,M)
    end
    println("test_vrr successfully completed")
end

test_vrr (generic function with 1 method)

In [39]:
function help_vrr(ax,ay,az,na,aI,aJ,aK,aexpn,bx,by,bz,nb,bexpn,
    cx,cy,cz,nc,cI,cJ,cK,cexpn,dx,dy,dz,nd,dexpn,M)
    # Helper function for vrr, since I for some reason changed the call signature
    # from the python version. This version has the same signature as the python version
    # and calls the Julia version
    na*nb*nc*nd*vrr(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
        cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,M)
end

function help_vrr_iter(ax,ay,az,na,aI,aJ,aK,aexpn,bx,by,bz,nb,bexpn,
    cx,cy,cz,nc,cI,cJ,cK,cexpn,dx,dy,dz,nd,dexpn,M)
    # Helper function for vrr, since I for some reason changed the call signature
    # from the python version. This version has the same signature as the python version
    # and calls the Julia version
    na*nb*nc*nd*vrr_iter(aexpn,ax,ay,az,aI,aJ,aK,bexpn,bx,by,bz,
        cexpn,cx,cy,cz,cI,cJ,cK,dexpn,dx,dy,dz,M)
end

help_vrr_iter (generic function with 1 method)

In [40]:
test_vrr()

test_vrr successfully completed


# Timing comparisons
Python/C times are from the int2e-speed.ipynb notebook in the same folder.

## TODO
* Get Python values for integrals for comparison
* Fix vrr_iter calls
* Look for time improvements

## s orbitals on the same center:
9 usec time Julia compared to 0.75 usec in C and 49 usec in Python

In [47]:
#                xyza   na   lmna   aa   xyzb    nb ab   xyzc      nc  lmnc   ac  xyzd   nd ad  M
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0)
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0)

  0.000013 seconds (51 allocations: 2.594 KiB)
  0.000011 seconds (51 allocations: 2.594 KiB)


4.373354581904758

In [48]:
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0)
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0)

  0.000124 seconds (81 allocations: 3.531 KiB)
  0.000121 seconds (81 allocations: 3.531 KiB)


4.373354581904758

## sp orbitals on the same center:
0.95 usec C, 58 usec Python, 13 usec Julia

In [51]:
#           xyza   na  lmna   aa  xyzb   nb ab  xyzc   nc  lmnc   ac  xyzd   nd ad  M
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 1,0,0, 1., 0.,0.,0., 1.,1., 0)
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 1,0,0, 1., 0.,0.,0., 1.,1., 0)

  0.000024 seconds (137 allocations: 7.328 KiB)
  0.000024 seconds (137 allocations: 7.328 KiB)


0.0

In [52]:
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 1,0,0, 1., 0.,0.,0., 1.,1., 0)
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,0., 1.,1., 0.,0.,0., 1., 1,0,0, 1., 0.,0.,0., 1.,1., 0)

  0.000130 seconds (95 allocations: 3.781 KiB)
  0.000139 seconds (95 allocations: 3.781 KiB)


0.0

## sp integrals, 4 different centers
1.2 usec C, 66 usec Python, 23 Julia

In [54]:
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,0,0, 1., 1.,1.,0., 1.,1., 0)
@time help_vrr(0.,0.,0., 1., 0,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,0,0, 1., 1.,1.,0., 1.,1., 0)

  0.000024 seconds (135 allocations: 7.281 KiB)
  0.000023 seconds (135 allocations: 7.281 KiB)


0.4746004027185156

In [55]:
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,0,0, 1., 1.,1.,0., 1.,1., 0)
@time help_vrr_iter(0.,0.,0., 1., 0,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,0,0, 1., 1.,1.,0., 1.,1., 0)

  0.000124 seconds (93 allocations: 3.734 KiB)
  0.000119 seconds (93 allocations: 3.734 KiB)


0.0

## pd integrals, 4 different centers
3 usec C, 95 usec Python, 32 usec Julia

In [59]:
@time help_vrr(0.,0.,0., 1., 1,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,1,0, 1., 1.,1.,0., 1.,1., 0)
@time help_vrr(0.,0.,0., 1., 1,0,0, 1., 0.,0.,1., 1.,1., 0.,1.,0., 1., 1,1,0, 1., 1.,1.,0., 1.,1., 0)

  0.000039 seconds (747 allocations: 40.594 KiB)
  0.000032 seconds (747 allocations: 40.594 KiB)


-0.01222864716176887